## PySpark SQL - Row, createDataFrame, createOrReplaceTempView

1. Row

- 데이터프레임의 한행을 표현하는 객체
- 딕셔너리와 비슷하지만, 만들고 나면 수정 불가능
- Row(키=값) , createDAtFrame할때 스파크가 알아서 스키마를 만들어준다

2. createDataFrame

- 데이터프레임 만들때 사용 

3. createOrReplaceTempView

- SQL을 쓰기 위해 테이블 이름 등록
- 현재 SparkSession안에서만 보인다
- 세션이 끝나면 spark.stop()하면 사라진다

<-> createGlobalTempView : 애플리케이션 종료시 사라짐

In [19]:
from pyspark.sql import (
    Row,
    SparkSession)

In [2]:
spark=(
    SparkSession
    .builder
    .appName("SQLSPARK")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 01:38:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc=spark.sparkContext

In [5]:
lines=sc.textFile("file:///workspace/data/income.txt")
lines.take(2)

['Kam Long|Dominica|VinThomas@example.taobao|137611',
 'Jamey Warner|Botswana|badlittleduck@test.gf|134999']

In [6]:
def parse_line(line:str):
    fields=line.split('|')
    return Row(
        name=str(fields[0]),
        country=str(fields[1]),
        email=str(fields[2]),
        compensation=int(fields[3])
    )

income_data=lines.map(parse_line)
income_data.take(2)

[Row(name='Kam Long', country='Dominica', email='VinThomas@example.taobao', compensation=137611),
 Row(name='Jamey Warner', country='Botswana', email='badlittleduck@test.gf', compensation=134999)]

In [9]:
schema_income=spark.createDataFrame(data=income_data).cache()
schema_income.show()

+-----------------+--------------------+--------------------+------------+
|             name|             country|               email|compensation|
+-----------------+--------------------+--------------------+------------+
|         Kam Long|            Dominica|VinThomas@example...|      137611|
|     Jamey Warner|            Botswana|badlittleduck@tes...|      134999|
|      Theola Page|              Malawi|   sharvin@test.mint|      171808|
|      Wes Simpson|        Turkmenistan|joshuasortino@exa...|       65429|
|  Loriann Hammond|           Argentina|osvaldas@example....|      183451|
|     Dannie Watts|Virgin Islands, B...|abdullindenis@tes...|      194254|
|    Darby Sanders|         Timor-Leste|suprb@example.xn-...|      195497|
| Willian Cummings|             Senegal|    areus@test.canon|       77369|
| Patricia Lindsey|              Bhutan|mikemai2awesome@t...|      141602|
|       Fae Howell|         Timor-Leste|wr@example.xn--3o...|      147314|
|  Annalee Pearson|      

In [12]:
schema_income.createOrReplaceTempView("income")

In [14]:
# 테이블 목록 확인
spark.catalog.listTables()

[Table(name='income', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
medium_income_df=spark.sql(
    """SELECT * FROM income
    WHERE compensation BETWEEN 70000 AND 100000"""
)
medium_income_df.show()

+------------------+--------------------+--------------------+------------+
|              name|             country|               email|compensation|
+------------------+--------------------+--------------------+------------+
|  Willian Cummings|             Senegal|    areus@test.canon|       77369|
|      Clarita Gill|             Ecuador| tomaslau@test.games|       86986|
| Walter Washington|          Kazakhstan|mbilderbach@examp...|       91072|
|       Lexie Banks|                Mali|unterdreht@test.date|       97933|
|        Luise Hunt|               Kenya|adellecharles@tes...|       96175|
|     Sebrina Walsh|         Puerto Rico|andrewcohen@examp...|       99276|
|      Josiah Lyons|              Malawi|nandini_m@test.ry...|       91768|
|      Temeka Grant|              Israel|terryxlife@test.g...|       71642|
|  Narcisa Saunders|Palestinian Terri...|raquelwilson@exam...|       77287|
|      Lisbeth Lane|          Azerbaijan|coreyweb@test.coffee|       82473|
|       Evan

In [20]:
spark.stop()